In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [ ]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
#submission

In [ ]:
Test = False
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
            for filename in filenames:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [ ]:
def get_feature_array(img_file_list):
    pbar.update(1)
    dicom_ds = dcmread(img_file_list)
    img_array = dicom_ds.pixel_array
    return np.resize(np.mean(img_array.T, axis=0),(128,))

vfunc = np.vectorize(get_feature_array, otypes=[object])

#### Snippet for test run only 

In [ ]:
"""rows = {}
global pbar 
pbar = 0
for st in test_dict.keys():
    with tqdm(total=len(test_dict[st])) as pbar:
        final_feature_list = vfunc(test_dict[st]).tolist()
    final = np.array(final_feature_list)
    y_proba = model.predict(final)
    rows[st] = np.mean(y_proba, axis=0)"""


In [ ]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        final_feature_list = vfunc(test_dict[st]).tolist()
                        final = np.array(final_feature_list)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])

In [ ]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

In [ ]:
submission.to_csv('submission.csv', index=False)